In [1]:
#source = https://techpolicyinstitute.org/publications/miscellaneous/rotten-tomatoes-and-imdb-reviews-strongly-correlated-with-movie-revenues/

## Scraping The Numbers: https://www.the-numbers.com/movie/budgets/all

In [8]:
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import duckdb
from urllib import parse
from urllib.parse import urljoin

In [9]:
#create fake user agents to use while requesting
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]

### Get pagination

In [10]:
#get request to the above url
url = 'https://www.the-numbers.com/movie/budgets/all/'
r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
# print(r.text)
soup = BeautifulSoup(r.content, "html.parser")

#class pagination to get to all the pages ("/movie/budgets/all" till "/movie/budgets/all/6301")
for i in soup.find_all(class_='pagination'):
    print(i.find_all('a'))


In [11]:
# Get pagination and add to list page_add
page_add = ['all']
nums = range(101,6302,100)
page_additions = list(nums)

for i in page_additions:  
    page_add.append(str(i))

#get all urls to scrape
# for i in page_add[1:]:
#     url = urljoin(url, i)
#     print('url=', url)

In [12]:
soup

<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<link href="/images/logo_2021/favicon.ico" rel="icon"/>
<meta content="nofollow, NOARCHIVE, NOODP, NOYDIR" name="robots"/>
<!-- Global site tag (gtag.js) - Google Analytics -->
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-5K2DT3XQN5"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-5K2DT3XQN5');
</script>
<!--<script async src="https://www.googletagmanager.com/gtag/js?id=UA-1343128-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1343128-1');
</script>-->
<meta content='(PICS-1.1 "https://www.icra.org/ratingsv02.html" l gen true for "https://www.the-numbers.com/" r (cb 1 lz 1 nz 1 oz 1 vz 1) "https://www.rsac.org/r

### Get dates

#### !! volgorde van de items wil niet... hieronder ook weer andere volgorde als tijdens het scrapen.. morgen verder naar kijken

In [6]:
# get date
date =[]
dates = soup.find_all('tr')
for tr in dates:
    cols = tr.find_all('td')
    for i in cols:
        t = cols[1].text[7:]
        date.append(t)

date

[]

## Scraping all pages

In [7]:
# Create empty dfs
df = pd.DataFrame()
dates_df = pd.DataFrame()
df_dates = pd.DataFrame()
df_budgets = pd.DataFrame()
df_names = pd.DataFrame()
budget_df = pd.DataFrame()
names_df =  pd.DataFrame()

for i in page_add:
    if i == 'all':
        url = 'https://www.the-numbers.com/movie/budgets/'
        url = urljoin(url, i)
    else:
        url = 'https://www.the-numbers.com/movie/budgets/all/'
        url = urljoin(url, i)

    #get request to the above url
    r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    # print(r.text)
    soup = BeautifulSoup(r.content, "html.parser")
    
    
    ### Dates: get acces to  all the dates
    #create a list with only the dates 
    dates = soup.find_all('tr')

    # loop over the list of movies to only keep the budgets and add them to the df
    year = []
    for tr in dates:
        cols = tr.find_all('td')
        for item2 in cols:
            y = cols[1].text[7:]
            year.append(y)               ######waarom pakt hij een andere volgorde??

    # add dates to dataframe with the position
    if i == 'all':
        dates_df['year'] = year
        dates_df['position'] = pd.Series(np.arange(1,100,1)) ###

    else:
        dates_df['year'] = year
        i = int(i)
        dates_df['position'] = pd.Series(np.arange(i,(i+100),1)) ###

    df_dates = pd.concat([df_dates, dates_df], ignore_index=True)

    #empty df again to start over and apend the new items of the next page in the next iterations
    dates_df = pd.DataFrame()
    

    ### BUDGETS: get acces to the data class which provides all the budgets etc
    #create a list with only the budgets per movie
    movie_budgets = soup.find_all(class_ = 'data')[1:399:4]

    # loop over the list of movies to only keep the budgets and add them to the df
    m_budget = []
    for a in movie_budgets:
        a = a.get_text()
        m_budget.append(a)

    # add budgets to dataframe with the position
    if i == 'all':
        budget_df['budget'] = m_budget
        budget_df['position'] = pd.Series(np.arange(1,100,1)) ###

    else:
        budget_df['budget'] = m_budget
        i = int(i)
        budget_df['position'] = pd.Series(np.arange(i,i+int(100),1)) ###

    df_budgets = pd.concat([df_budgets, budget_df], ignore_index=True)

    #empty df again to start over and apend the new items of the next page in the next iterations
    budget_df = pd.DataFrame()

    
    ### MOVIE NAMES
    #get the name of the movies in b element in a list
    movie_names = []
    for a in soup.findAll('b'):
        movie_names.append(a.text)

    #############
    # loop over the list of movienames to only keep the nam add them to the df
    m_names= []
    for name in movie_names[1:-3]:
        name = str(name)
        m_names.append(name)

    # add budgets to dataframe with the position
    if i == 'all':
        names_df['title'] = m_names
        names_df['position'] = pd.Series(np.arange(1,100,1)) ###
    else:
        names_df['title'] = m_names
        i = int(i)
        names_df['position'] = pd.Series(np.arange(i,i+int(100),1)) ###

    df_names = pd.concat([df_names, names_df], ignore_index=True)

    #empty df again to start over and apend the new items of the next page in the next iterations
    names_df = pd.DataFrame()
    #############
    #merge on position: df_budgets and df_names
    
    df = pd.merge(df_names, df_budgets, on='position')
    df2 = pd.merge(df, df_dates, on='position')
#     df_names.merge(df_budgets,on='position').merge(df_dates,on='position') ##hier lijkt het ook niet aan te liggen?

df_dates.head(25)


,year,position
0,2022,1.0
1,2022,2.0
2,2022,3.0
3,2022,4.0
4,2022,5.0
5,2022,6.0
6,2019,7.0
7,2019,8.0
8,2019,9.0
9,2019,10.0


### Dataframe with the Scraped information ( including names and budgets)

In [85]:
# df = pd.to_datetime(df['Date'])
# df['year'] = pd.to_datetime(df['year'], errors='coerce')
df2

,title,position,budget,year
0,Avatar: The Way of Water,1.0,"$460,000,000",2022
1,Avengers: Endgame,2.0,"$400,000,000",2022
2,Pirates of the Caribbean: On Stranger Tides,3.0,"$379,000,000",2022
3,Avengers: Age of Ultron,4.0,"$365,000,000",2022
4,Fast X,5.0,"$340,000,000",2022
...,...,...,...,...
38206,Following,6369.0,"$6,000",2013
38207,Return to the Land of Wonders,6370.0,"$5,000",2013
38208,A Plague So Pleasant,6371.0,"$1,400",2013
38209,My Date With Drew,6372.0,"$1,100",2013


### Clean names so we can merge with the project data

In [13]:
def convert_word(word):
    """
    Strips a word from all punctuation, whitespace, and digits. Then converts the word into all lower case
    and convert all characters with accents to non-accent letters.
    """
    word = re.sub(r'[^\w\s]','',word).lower()
    word = re.sub(r'[àáâãäå]', 'a', word)
    word = re.sub(r'[èéêë]', 'e', word)
    word = re.sub(r'[ìíîï]', 'i', word)
    word = re.sub(r'[òóôõö]', 'o', word)
    word = re.sub(r'[ùúûü]', 'u', word)
    
    return word

In [14]:
def map_cleaned_strings(dfname, columnname):
    """
    Maps new values in a df-column. From a new created dictionary, with as Key the 'old' string value and 
    as Value the 'new'-string value. The 'new'-string values are extracted with the function convert_word().
    """
    dict_old_new = {}
    for i in dfname[columnname]:
        if type(i) != str:
            continue
        else:
            dict_old_new[i] = convert_word(i)

    dfname[columnname] = dfname[columnname].map(dict_old_new)
    
    return dfname[columnname]

In [15]:
#call function map_cleaned_strings to apply in the dataframe on the column primary title
map_cleaned_strings(df, 'title')

0                          avatar the way of water
1                                 avengers endgame
2       pirates of the caribbean on stranger tides
3                           avengers age of ultron
4                                           fast x
                           ...                    
6368                                     following
6369                 return to the land of wonders
6370                          a plague so pleasant
6371                             my date with drew
6372                                        neeras
Name: title, Length: 6373, dtype: object

In [16]:
df

,title,position,budget
0,avatar the way of water,1.0,"$460,000,000"
1,avengers endgame,2.0,"$400,000,000"
2,pirates of the caribbean on stranger tides,3.0,"$379,000,000"
3,avengers age of ultron,4.0,"$365,000,000"
4,fast x,5.0,"$340,000,000"
...,...,...,...
6368,following,6369.0,"$6,000"
6369,return to the land of wonders,6370.0,"$5,000"
6370,a plague so pleasant,6371.0,"$1,400"
6371,my date with drew,6372.0,"$1,100"


### Add as table to database

In [17]:
# create a connection to a file called 'imdb_reviews.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it
con.sql("CREATE TABLE IF NOT EXISTS budget_data AS SELECT * FROM df")

con.close()